In [2]:
from flask import Flask, jsonify
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import re
import demoji
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import sqlite3

In [3]:
df_kbbi = pd.read_csv('new_kamusalay.csv', header=None, encoding='ISO-8859-1', names=['TIDAKBAKU', 'BAKU'])

ABS = pd.read_csv('abusive.csv')

data = pd.read_csv('data.csv', header=None, encoding='latin-1')

data2 = data[[0]]


In [8]:
def lowercase(text):
    return text.lower()

def removechars(text):
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub('rt',' ',text) 
    text = re.sub('user',' ',text) 
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text)
    # text = re.sub('xf0',' ',text) 
    # text = re.sub('x9f',' ',text) 
    # text = re.sub('x98',' ',text) 
    # text = re.sub('x84',' ',text) 
    return text
    

def remove_emoji(text):
    dem = demoji.findall(text)
    for demoj in dem.keys():
        text = text.replace(demoj, '')
    return text

def sensor(text):
    abusiveword = ABS['ABUSIVE'].tolist()
    for word in abusiveword:
        pattern = re.compile(r'\b{}\b'.format(word))
        length = len(word)
        replacement = '*' * length
        text = pattern.sub(replacement, text.lower())
    return text

def changealay(text):
    alay = dict(zip(df_kbbi['TIDAKBAKU'], df_kbbi['BAKU']))
    text = ' '.join([alay[word] if word in alay else word for word in text.split(' ')])
    return text

In [9]:
dataw = (removechars('RT rt Rt user abc !@# ?/\ ABV 0123 /n 🥹 😂 🤣 🥲'))
dataw

'RT   Rt   abc         ABV 0123  n        '

In [5]:
print(lowercase('HHALO, Apa Kabar?'))

hhalo, apa kabar?


In [7]:
print(sensor('anjing'))

******


In [11]:
print(remove_emoji('🥲asu'))

asu


In [8]:
print(changealay('gws'))

semoga lekas sembuh


In [13]:
def cleaning(text):
    text = removechars(text)
    text = lowercase(text)
    text = changealay(text)
    text = sensor(text)
    text = remove_emoji(text)
    return text

In [14]:
print(cleaning('Halo, user ANJING RT gws Apa Kabar 😂 🤣 🥲?'))


halo    ****** rt semoga lekas sembuh apa kabar       


In [12]:
data1 = data2[0].apply(cleaning)


In [ ]:
data1 = data2[0].apply(cleaning)

conn = sqlite3.connect('datacsv_tosql1.db')
data1.to_sql('Datanew', conn, if_exists='replace', index=False)

conn.close()

In [13]:
data1.to_csv('cleaned1.csv', index=False, header=False)

In [14]:
conn = sqlite3.connect('datacsv_tosql1.db')
data1.to_sql('Datanew', conn, if_exists='replace', index=False)

conn.close()

In [15]:
cnx = sqlite3.connect('datacsv_tosql.db')
sql_df = pd.read_sql_query('Select * from Datanew', cnx)
sql_df

,0
0,twit
1,di saat semua cowok berusaha melacak perhati...
2,pengguna pengguna siapa yang telat memberi ...
3,41 kadang aku berpikir kenapa aku tetap perc...
4,pengguna pengguna aku itu aku dan ku tau matam...
...,...
13165,pengguna jangan asal berbicara ndasmu ...
13166,pengguna kasur mana enak
13167,pengguna hati hati tidak dan lagi bos...
13168,pengguna pengguna pengguna pengguna bom yang r...


In [16]:
dataq = pd.read_csv('cleaned1.csv', header=None, encoding='latin-1')

In [17]:
dataq

,0
0,twit
1,di saat semua cowok berusaha melacak perhati...
2,pengguna pengguna siapa yang telat memberi ...
3,41 kadang aku berpikir kenapa aku tetap perc...
4,pengguna pengguna aku itu aku dan ku tau matam...
...,...
13165,pengguna jangan asal berbicara ndasmu ****** ...
13166,pengguna kasur mana enak ******
13167,pengguna hati hati **** tidak dan lagi bos...
13168,pengguna pengguna pengguna pengguna bom yang r...
